In [ ]:
%pip install -U langchain langchain-community faiss-cpu openai unstructured pymupdf python-docx \
    langchain-openai sentence_transformers numpy pandas networkx openpyxl \
    langchain-huggingface


  Using cached faiss_cpu-1.11.0.post1-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
Using cached faiss_cpu-1.11.0.post1-cp312-cp312-win_amd64.whl (14.9 MB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------

In [ ]:
# conda install -c conda-forge faiss-gpu cudatoolkit=11.6

  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached scipy-1.16.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupS

ERROR: Could not install packages due to an OSError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'c:\\Users\\iwcyou\\.conda\\envs\\dev\\Lib\\site-packages\\mpmath\\tests\\test_functions.py'
Consider using the `--user` option or check the permissions.



In [1]:
from langchain.vectorstores import FAISS
import torch
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh",  # 或其他
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# from langchain_openai import OpenAIEmbeddings #需要openai的 API Key
# 如果需要使用 HuggingFace 的模型
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    PyPDFLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredExcelLoader,
)
import os

# 支持加载多种格式的文档
def load_documents_from_folder(folder_path: str):
    docs = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif filename.endswith(".docx"):
            loader = UnstructuredWordDocumentLoader(file_path)
        elif filename.endswith(".xlsx") or filename.endswith(".xls"):
            loader = UnstructuredExcelLoader(file_path)
        else:
            continue
        docs.extend(loader.load())
    return docs

# 切块 + 嵌入 + 存入 FAISS
def build_faiss_index_from_folder(folder_path: str, index_save_path: str):
    print(" 加载法规文档中...")
    raw_docs = load_documents_from_folder(folder_path)
    
    print(" 切分文档为片段...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    split_docs = splitter.split_documents(raw_docs)
    
    print(" 构建嵌入向量...")
    # embeddings = OpenAIEmbeddings()  # 或 HuggingFaceEmbeddings(model_name=\"...\")
    embeddings = embedding_model  

    print(" 构建 FAISS 向量数据库...")
    vectordb = FAISS.from_documents(split_docs, embeddings)

    print(f" 保存向量数据库至：{index_save_path}")
    vectordb.save_local(index_save_path)
    return vectordb


c:\Users\iwcyou\.conda\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 示例调用
folder = "./rule"
save_path = "./faiss_law_index"
build_faiss_index_from_folder(folder, save_path)

 加载法规文档中...
 切分文档为片段...
 构建嵌入向量...
 构建 FAISS 向量数据库...
 保存向量数据库至：./faiss_law_index


In [8]:
def query_law_index(query_text: str, index_path: str = "./faiss_law_index", k=3):
    vectordb = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True
)
    docs = vectordb.similarity_search(query_text, k=k)
    return "\n\n".join([d.page_content for d in docs])


In [9]:
state = {
    'filled_input': '运输企业在运输过程中，存在泥头车路面遗洒，责任如何划分？'
}
rag_context = query_law_index(state['filled_input'])

prompt = f"""
请参考以下法规内容作为决策依据:
{rag_context}

描述如下:
{state['filled_input']}

请判断责任归属及依据。
输出：
{{
  "responsibility": "运输企业",
  "reason": "......"
}}
"""


In [10]:
from langchain.chat_models import ChatOpenAI  # 可替换为 deepseek-r1
from langchain.schema import HumanMessage
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# ✅ 加载 FAISS 向量库
vectordb = FAISS.load_local(
    folder_path="./faiss_law_index",  # 你保存的知识库路径
    embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh"),
    allow_dangerous_deserialization=True
)

# ✅ 构造检索函数
def query_law_index(query_text: str, k=3):
    docs = vectordb.similarity_search(query_text, k=k)
    return "\n\n".join([doc.page_content for doc in docs])

# ✅ 构造 LangChain 模型（可替换为 DeepSeek）
llm = ChatOpenAI(temperature=0, model="gpt-4")  # 或调用 deepseek-r1 的兼容封装

# ✅ 模拟状态输入
state = {
    'filled_input': '运输企业在运输过程中，存在泥头车路面遗洒，责任如何划分？'
}

# ✅ 查询法规知识库
rag_context = query_law_index(state['filled_input'])

# ✅ 构造最终 Prompt
prompt = f"""
你是一位城市治理法规专家，请结合以下法规内容判断城市泥头车污染事件的责任归属。

法规内容如下：
{rag_context}

描述如下：
{state['filled_input']}

请判断责任归属，并说明理由。
输出 JSON 格式：
{{
  "responsibility": "运输企业",
  "reason": "......"
}}
"""

# ✅ 调用大模型进行回答
response = llm.invoke([HumanMessage(content=prompt)])

# ✅ 输出结果
print("📘 模型返回：")
print(response.content)


C:\Users\iwcyou\AppData\Local\Temp\ipykernel_14348\2373740099.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model="gpt-4")  # 或调用 deepseek-r1 的兼容封装


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <your-op*********key>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}